In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [13]:
def delta_days_ago_and_current(tickers, dates, delta=7):
    """This function obtains, for each pair of ticker and date, the closing price of the ticker delta days
    before the given date and the closing price of the ticker for the day of the reference date.
    
    For the inputs:
    tickers: List of tickers, each represented by a string. Same length as dates!
    dates: List of dates, each represented in the format %Y-%m-%d (e.g. 2010-01-24)
    delta: Number of days before the reference date from which to obtain the previous price. It can also be a list,
        with as many deltas as desired.
    
    The output is a pandas dataframe, with as many rows as specified tickers, and columns Reference Date, 
    Previous Close, and Current Close."""
    
    if type(delta) == int:
        delta = [delta]
    
    results = {field: [] for field in ['Ticker', "Reference Date", "Current Close"] + [f"Previous Close (delta={d})" for d in delta]}
    
    for i, t in enumerate(tickers):
        #If date falls in weekends, take Friday and Monday
        extra_add = 0
        if datetime.strptime(dates[i], '%Y-%m-%d').isoweekday() == 6:
            extra_add = -1
        elif datetime.strptime(dates[i], '%Y-%m-%d').isoweekday() == 7:
            extra_add = 1
        
        current = datetime.strptime(dates[i], '%Y-%m-%d') + timedelta(days=extra_add)
        
        if max(delta) >= 0:
            max_previous = current + timedelta(days=-max(delta))
            if min(delta) > 0:
                max_next = current
            else:
                max_next = current + timedelta(days=-min(delta))    
        else:
            max_next = current + timedelta(days=-min(delta)) 
            max_previous = current
            
        data = yf.download(t, start=max_previous + timedelta(days=-2), end=max_next + timedelta(days=2))
        current_close = data.iloc[-1]['Close']
        
        results['Ticker'].append(t)
        results["Reference Date"].append(current)
        results["Current Close"].append(current_close)
        for d in delta:
            previous = current + timedelta(days=-d)
            
            #If date falls in weekends, take Friday and Monday
            if previous.isoweekday() == 6:
                previous += timedelta(days=-1)
            elif previous.isoweekday() == 7:
                previous += timedelta(days=+1)
            
            previous_close = data.loc[data.index == previous, 'Close'][0]
            results[f"Previous Close (delta={d})"].append(previous_close)

    results = pd.DataFrame(results).set_index('Ticker')
    return results

In [14]:
tickers = ['AAPL', 'GOOG']
dates = ['2012-03-15', '2010-08-21']
deltas = [7, 5, 3, -1, -2, -5]
df = delta_days_ago_and_current(tickers, dates, deltas)
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Reference Date,Current Close,Previous Close (delta=7),Previous Close (delta=5),Previous Close (delta=3),Previous Close (delta=-1),Previous Close (delta=-2),Previous Close (delta=-5)
Ticker,,,,,,,,
AAPL,2012-03-15,21.517857,19.356787,19.470358,19.714287,20.913214,20.913214,21.641430
GOOG,2010-08-20,224.647964,242.266922,241.888336,244.344131,230.147354,231.168518,226.461166
